In [8]:
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# サンプルデータの生成（実際のデータに置き換えてください）
X = np.random.rand(100, 10)  # 100サンプル、10個の特徴量
y = np.random.rand(100)

# パイプラインの定義（正規化とSVR）
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

# ハイパーパラメータの範囲を定義
param_grid = {
    'svr__C': [0.1, 1, 10, 100],
    'svr__epsilon': [0.01, 0.1, 0.5],
    'svr__kernel': ['rbf', 'linear'],
    'svr__gamma': ['scale', 'auto', 0.1, 1]
}

# 交差検証の設定
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCVの設定
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)

# グリッドサーチの実行
grid_search.fit(X, y)

# 最適なパラメータと最良のスコアを表示
print("Best parameters:", grid_search.best_params_)
print("Best negative MSE:", grid_search.best_score_)

# 最適なモデルを取得
best_model = grid_search.best_estimator_

# 交差検証を使用して最終的な性能を評価
cv_scores = cross_val_score(best_model, X, y, cv=cv, scoring='neg_mean_squared_error')
mse_scores = -cv_scores  # 負の MSE を正の MSE に変換
r2_scores = cross_val_score(best_model, X, y, cv=cv, scoring='r2')

print(f"Cross-validation MSE: {mse_scores.mean():.4f} (+/- {mse_scores.std() * 2:.4f})")
print(f"Cross-validation R-squared: {r2_scores.mean():.4f} (+/- {r2_scores.std() * 2:.4f})")

# 全データセットでモデルを再学習
best_model.fit(X, y)

# 予測を行い、最終的なスコアを計算
y_pred = best_model.predict(X)
final_mse = mean_squared_error(y, y_pred)
final_r2 = r2_score(y, y_pred)

print(f"Final MSE on full dataset: {final_mse:.4f}")
print(f"Final R-squared on full dataset: {final_r2:.4f}")

Best parameters: {'svr__C': 0.1, 'svr__epsilon': 0.5, 'svr__gamma': 'scale', 'svr__kernel': 'rbf'}
Best negative MSE: -0.08080588656000673
Cross-validation MSE: 0.0808 (+/- 0.0209)
Cross-validation R-squared: -0.0272 (+/- 0.0600)
Final MSE on full dataset: 0.0811
Final R-squared on full dataset: -0.0000
